### IMPORTATION DES BIBLIOTHEQUES

In [400]:
import pandas as pd
import numpy as np

### IMPORTATION DES DONNEES

In [513]:
fichier = pd.read_csv("D:/Users/11561/Downloads/cash_cobra/journaux_caisse.csv",sep=",")

In [530]:
fichier.head()

,Agence,Numero caisse,Date,Hour,Eve,Compte,Ope,Libelle,Debit,Credit,day,month,year,Code agence,Periode,TypeOperation,profil
0,BANQUE PATRIMONIALE,1,2021-11-02,23:44,60049,001403993 - 85,119,RETRAIT CHEQUE CLIENT AE-9920729 ACKAH JEAN MA...,0,4000000,2,11,2021,1,pic,RETRAIT,1.0
1,BANQUE PATRIMONIALE,1,2021-11-02,23:44,60133,111053578 - 45,119,RETRAIT CHEQUE CLIENT AE-9691981 OUSMANE COULI...,0,1500000,2,11,2021,1,pic,RETRAIT,1.0
2,AGENCE PRIVILEGE,1,2021-11-02,23:44,64825,075155343 - 57,119,RETRAIT CHEQUE CLIENT AD-1334571 DE SOUZA OLAT...,0,5000000,2,11,2021,75,pic,RETRAIT,1.0
3,AGENCE PRIVILEGE,1,2021-11-02,23:44,536605,111050309 - 74,90,RETRAIT CHEQUE 02582888 ELLOH RITA CATHERINE E...,0,1000000,2,11,2021,75,pic,RETRAIT,1.0
4,AGENCE PRIVILEGE,1,2021-11-02,23:44,536606,075402521 - 79,90,RETRAIT CHEQUE 02582889 SIDIKI CISSE,0,5000000,2,11,2021,75,pic,RETRAIT,1.0


### TRAITEMENT DES VALEURS MANQUANTES ET CONVERTION DES VARIABLES EN ENTIERS

In [515]:
fichier['Credit']= fichier['Credit'].str.replace(r'\D+',"", regex=True)
fichier['Debit' ]= fichier['Debit' ].str.replace(r'\D+',"", regex=True)
fichier['Debit' ] = fichier['Debit'].fillna(0)
fichier['Credit'] = fichier['Credit'].fillna(0)
fichier['Credit'] = list(map(int,fichier['Credit']))
fichier['Debit'] = list(map(int,fichier['Debit']))

### CREATION DES BUCKETS DE MONTANT

In [436]:
fichier['CreditDebit'] = fichier['Credit'].values
fichier.loc[fichier['CreditDebit']==0,'CreditDebit'] = fichier.loc[fichier['CreditDebit']==0,'Debit'].values

In [437]:
### FONCTION POUR CREER LES BUCKETS DE MONTANTS

def CreaBucket(x):
    fichier['Bucket'] = 0
    for i in range(0,fichier.shape[0]):
        if fichier[x][i]==0 :
            fichier['Bucket'][i] = "vide"
        elif fichier[x][i]>1 and fichier[x].iloc[i]<= 300000:
            fichier['Bucket'][i] = "[1 - 300 000]"
        elif fichier[x][i]> 300000 and fichier[x][i]<= 600000:
            fichier['Bucket'][i] = "[300 001 - 600 000]"
        elif fichier[x][i]> 600001 and fichier[x][i]<= 1000000:
            fichier['Bucket'][i] = "[600 001 - 1 000 000]"
        elif fichier[x][i]>1000001 and fichier[x][i]<= 5000000:
            fichier['Bucket'][i] = "[1 000 001 - 5 000 000]"
        elif fichier[x][i]>5000001 and fichier[x][i]<= 10000000:
            fichier['Bucket'][i] = "[5 000 001 - 10 000 000]"
        else:
            fichier['Bucket'][i] = "[10 000 000 et plus]"
    return fichier

In [438]:
# fichier = CreaBucket('CreditDebit')

### CREATION DES PERIODES (PIC ET CREUSE) ET TYPES D'OPERATION (RETRAIT ET VERSEMENT)

In [516]:
fichier['Periode']='pic'
fichier['TypeOperation']='RETRAIT'
fichier.loc[fichier['day'].isin(list(range(6,25))),'Periode']='creuse'
fichier.loc[fichier['Credit']==0,'TypeOperation']='VERSEMENT'

### PROFIL DE CHAQUE AGENCE

In [517]:
# profil = fichier.drop_duplicates(subset=['Agence','Numero caisse']).groupby('Agence').size().reset_index()
# profil.rename(index=str,columns={0:'profil'},inplace=True)
# fichier = fichier.join(profil.set_index('Agence'),on='Agence')
# fichier.drop(columns=['CreditDebit'],inplace=True)

In [518]:
profil  = pd.read_excel("D:/Users/11561/Downloads/cash_cobra/Requalification caisse By SQORP.xlsx")
profil.rename(index=str,columns={'Étiquettes de lignes':'Agence'}, inplace = True)

In [442]:
profil.profil.unique()

array([1, 2, 3, 4], dtype=int64)

In [519]:
fichier = fichier.join(profil.set_index('Agence'),on='Agence')
fichier.head()

,Agence,Numero caisse,Date,Hour,Eve,Compte,Ope,Libelle,Debit,Credit,day,month,year,Code agence,Periode,TypeOperation,Nombre de caisse selon SIRAMBANA,Proposition SQORP,profil
0,BANQUE PATRIMONIALE,1,2021-11-02,23:44,60049,001403993 - 85,119,RETRAIT CHEQUE CLIENT AE-9920729 ACKAH JEAN MA...,0,4000000,2,11,2021,1,pic,RETRAIT,1 caisse,1 caisse,1.0
1,BANQUE PATRIMONIALE,1,2021-11-02,23:44,60133,111053578 - 45,119,RETRAIT CHEQUE CLIENT AE-9691981 OUSMANE COULI...,0,1500000,2,11,2021,1,pic,RETRAIT,1 caisse,1 caisse,1.0
2,AGENCE PRIVILEGE,1,2021-11-02,23:44,64825,075155343 - 57,119,RETRAIT CHEQUE CLIENT AD-1334571 DE SOUZA OLAT...,0,5000000,2,11,2021,75,pic,RETRAIT,1 caisse,1 caisse,1.0
3,AGENCE PRIVILEGE,1,2021-11-02,23:44,536605,111050309 - 74,90,RETRAIT CHEQUE 02582888 ELLOH RITA CATHERINE E...,0,1000000,2,11,2021,75,pic,RETRAIT,1 caisse,1 caisse,1.0
4,AGENCE PRIVILEGE,1,2021-11-02,23:44,536606,075402521 - 79,90,RETRAIT CHEQUE 02582889 SIDIKI CISSE,0,5000000,2,11,2021,75,pic,RETRAIT,1 caisse,1 caisse,1.0


In [522]:
fichier.drop(columns=['Nombre de caisse selon SIRAMBANA','Proposition SQORP'],inplace=True)
fichier.head()

,Agence,Numero caisse,Date,Hour,Eve,Compte,Ope,Libelle,Debit,Credit,day,month,year,Code agence,Periode,TypeOperation,profil
0,BANQUE PATRIMONIALE,1,2021-11-02,23:44,60049,001403993 - 85,119,RETRAIT CHEQUE CLIENT AE-9920729 ACKAH JEAN MA...,0,4000000,2,11,2021,1,pic,RETRAIT,1.0
1,BANQUE PATRIMONIALE,1,2021-11-02,23:44,60133,111053578 - 45,119,RETRAIT CHEQUE CLIENT AE-9691981 OUSMANE COULI...,0,1500000,2,11,2021,1,pic,RETRAIT,1.0
2,AGENCE PRIVILEGE,1,2021-11-02,23:44,64825,075155343 - 57,119,RETRAIT CHEQUE CLIENT AD-1334571 DE SOUZA OLAT...,0,5000000,2,11,2021,75,pic,RETRAIT,1.0
3,AGENCE PRIVILEGE,1,2021-11-02,23:44,536605,111050309 - 74,90,RETRAIT CHEQUE 02582888 ELLOH RITA CATHERINE E...,0,1000000,2,11,2021,75,pic,RETRAIT,1.0
4,AGENCE PRIVILEGE,1,2021-11-02,23:44,536606,075402521 - 79,90,RETRAIT CHEQUE 02582889 SIDIKI CISSE,0,5000000,2,11,2021,75,pic,RETRAIT,1.0


In [521]:
fichier.shape

(963409, 19)

### Regroupement journalier du nombre de transaction

In [523]:
tous = fichier.groupby(['Agence','day','month','year','TypeOperation','profil','Periode']).size().reset_index().rename(index=str, columns={0:'NombreTransaction'})

In [524]:
tous.groupby(['Agence','TypeOperation','Periode'])['NombreTransaction'].mean().reset_index().head(50)

,Agence,TypeOperation,Periode,NombreTransaction
0,ABATTA,RETRAIT,creuse,50.800000
1,ABATTA,RETRAIT,pic,77.620690
2,ABATTA,VERSEMENT,creuse,73.260000
3,ABATTA,VERSEMENT,pic,72.172414
4,ABENGOUROU,RETRAIT,creuse,79.725490
5,ABENGOUROU,RETRAIT,pic,102.200000
6,ABENGOUROU,VERSEMENT,creuse,103.098039
7,ABENGOUROU,VERSEMENT,pic,97.366667
8,ABOBO GARE,RETRAIT,creuse,60.117647
9,ABOBO GARE,RETRAIT,pic,87.666667


In [528]:
agence = tous.groupby(['Agence','TypeOperation','profil','Periode'])['NombreTransaction'].mean().reset_index()
agence = agence.merge(s, on=['profil','TypeOperation','Periode'], how ='right')

In [529]:
agence

,Agence,TypeOperation,profil,Periode,NombreTransaction,seuil
0,ABATTA,RETRAIT,1.0,creuse,50.800000,33.836889
1,ABOBO SAMAKE,RETRAIT,1.0,creuse,32.549020,33.836889
2,ADZOPE,RETRAIT,1.0,creuse,27.176471,33.836889
3,AGENCE AKWABA,RETRAIT,1.0,creuse,20.117647,33.836889
4,AGENCE ANYAMA,RETRAIT,1.0,creuse,29.909091,33.836889
...,...,...,...,...,...,...
315,SAN-PEDRO,VERSEMENT,4.0,creuse,135.882353,93.383117
316,YOPOUGON BEL AIR,VERSEMENT,4.0,creuse,78.647059,93.383117
317,AUTOROUTE,VERSEMENT,4.0,pic,79.000000,100.477273
318,SAN-PEDRO,VERSEMENT,4.0,pic,130.766667,100.477273


### LES SEUILS

In [502]:
# columns=[]
# for a in tous['profil'].unique():
#     for b in tous['TypeOperation'].unique():
#         columns.append([a,b])
# columns

In [525]:
s = tous.groupby(['profil','TypeOperation','Periode'])['NombreTransaction'].mean().reset_index().rename(index=str, columns={'NombreTransaction':'seuil'})
s

,profil,TypeOperation,Periode,seuil
0,1.0,RETRAIT,creuse,33.836889
1,1.0,RETRAIT,pic,45.851636
2,1.0,VERSEMENT,creuse,44.833450
3,1.0,VERSEMENT,pic,46.363202
4,2.0,RETRAIT,creuse,66.031388
5,2.0,RETRAIT,pic,90.004726
6,2.0,VERSEMENT,creuse,74.304515
7,2.0,VERSEMENT,pic,75.156900
8,3.0,RETRAIT,creuse,100.321492
9,3.0,RETRAIT,pic,130.075710


In [483]:
# df = pd.DataFrame(columns, columns = ['profil','TypeOperation'])
# for i in list(tous.profil.unique()):
#     for j in list(tous.TypeOperation.unique()):
#         s =tous[(tous['profil']==i)&(tous['TypeOperation']==j)]['NombreTransaction'].mean()
#         df.loc[(df['profil']==i), 'seuil']= s
# df

,profil,TypeOperation,seuil
0,1.0,RETRAIT,45.400000
1,1.0,VERSEMENT,45.400000
2,2.0,RETRAIT,74.618302
3,2.0,VERSEMENT,74.618302
4,3.0,RETRAIT,101.390909
5,3.0,VERSEMENT,101.390909
6,4.0,RETRAIT,95.962810
7,4.0,VERSEMENT,95.962810


### création de ligne (Nombre de transaction 0) pour les seuils

In [294]:
teste = tous.copy()
teste['Id']=teste['Agence']+'_'+teste['day'].astype(str)+'_'+teste['month'].astype(str)+'_'+teste['year'].astype(str)+'_'+teste['Bucket']
teste['date'] = teste['day'].astype(str)+'-'+teste['month'].astype(str)+'-'+teste['year'].astype(str)
teste['date'] = pd.to_datetime(teste['date'], format='%d-%m-%Y')

In [295]:
date_min = teste.groupby(['Agence'])['date'].min().reset_index()

In [296]:
a = teste.drop_duplicates(subset =['Agence','Numero caisse'])
a = a[['Agence','Numero caisse']]

In [297]:
import datetime
columns=[]

for b in teste['Bucket'].unique():
    for p in teste['Periode'].unique():
        for t in teste['TypeOperation'].unique():
            for y in teste['date'].unique():
                columns.append([b,p,t,y])
frames=pd.DataFrame()
for d in columns:
    if columns.index(d)%500==0:
        print(columns.index(d))
    aux=a.copy()
    aux['Bucket']=d[0]
    aux['Periode']=d[1]
    aux['TypeOperation']=d[2]
    aux['date']=d[3]
    frames=pd.concat([frames,aux])

0
500
1000
1500
2000


In [298]:
b = teste.merge(frames , on =['Agence','date','Bucket','Periode','TypeOperation','Numero caisse'], how='outer')
b['NombreTransaction']=b['NombreTransaction'].fillna(0)
b = b.merge(date_min, on = 'Agence')

In [299]:
b.drop(columns = ['Id','profil'], inplace = True)
b['day']=b['date_x'].dt.day
b['month']=b['date_x'].dt.month
b['year']=b['date_x'].dt.year
b = b.join(profil.set_index('Agence'),on='Agence')

In [300]:
b.drop(columns=['date_x','date_y','Nombre de caisse selon SIRAMBANA','Proposition SQORP'], inplace=True)

In [318]:
b

,Agence,Numero caisse,day,month,year,Bucket,Periode,TypeOperation,NombreTransaction,profil
0,ABATTA,1,1,2,2022,[1 - 300 000],pic,RETRAIT,7.0,1
1,ABATTA,1,1,2,2022,[1 - 300 000],pic,VERSEMENT,8.0,1
2,ABATTA,1,1,2,2022,[1 000 001 - 5 000 000],pic,RETRAIT,5.0,1
3,ABATTA,1,1,2,2022,[1 000 001 - 5 000 000],pic,VERSEMENT,2.0,1
4,ABATTA,1,1,2,2022,[10 000 000 et plus],pic,VERSEMENT,1.0,1
...,...,...,...,...,...,...,...,...,...,...
365155,YOPOUGON ZONE INDUSTRIELLE,2,14,2,2022,[600 001 - 1 000 000],creuse,VERSEMENT,0.0,2
365156,YOPOUGON ZONE INDUSTRIELLE,3,14,2,2022,[600 001 - 1 000 000],creuse,VERSEMENT,0.0,2
365157,YOPOUGON ZONE INDUSTRIELLE,1,6,12,2021,[600 001 - 1 000 000],creuse,VERSEMENT,0.0,2
365158,YOPOUGON ZONE INDUSTRIELLE,2,6,12,2021,[600 001 - 1 000 000],creuse,VERSEMENT,0.0,2


### POURCENTAGE D'UTILISATION DE CHAQUE CAISSE PAR AGENCE

In [531]:
caisse_jour = fichier.drop_duplicates(subset = ['Agence','Numero caisse','Date']).groupby(['Agence','Date']).size().reset_index().rename(index=str,columns={0:'Nombre caisse'})
test = caisse_jour.groupby(['Agence','Nombre caisse']).size().reset_index().rename(index=str,columns={0:'nbre'})
for i in test['Agence'].unique():
    ag = test.groupby(test['Agence']==i)['nbre'].sum().reset_index()['nbre'][1]
    test.loc[test['Agence']==i,'%'] = (test.loc[test['Agence']==i]['nbre']/ag)*100
test = test.join(profil.set_index('Agence'),on='Agence')
test

,Agence,Nombre caisse,nbre,%,Nombre de caisse selon SIRAMBANA,Proposition SQORP,profil
0,ABATTA,1,47,59.493671,2 caisses,1 caisse,1.0
1,ABATTA,2,32,40.506329,2 caisses,1 caisse,1.0
2,ABENGOUROU,1,2,2.469136,3 caisses,2 caisses,2.0
3,ABENGOUROU,2,53,65.432099,3 caisses,2 caisses,2.0
4,ABENGOUROU,3,26,32.098765,3 caisses,2 caisses,2.0
...,...,...,...,...,...,...,...
167,YOPOUGON NIANGON SUD,2,26,32.500000,2 caisses,1 caisse,1.0
168,YOPOUGON SAINT ANDRE,1,3,3.750000,2 caisses,2 caisses,2.0
169,YOPOUGON SAINT ANDRE,2,77,96.250000,2 caisses,2 caisses,2.0
170,YOPOUGON ZONE INDUSTRIELLE,1,21,26.582278,3 caisses,2 caisses,2.0


In [2]:
import pandas as pd
test = pd.read_excel('d:/Users/11561/Downloads/cash_cobra/percentUtilisation.xlsx')


,Agence,Nombre caisse,nbre,%,profil
0,ABATTA,1,47,59.493671,2
1,ABATTA,2,32,40.506329,2
2,ABENGOUROU,1,2,2.469136,3
3,ABENGOUROU,2,53,65.432099,3
4,ABENGOUROU,3,26,32.098765,3
...,...,...,...,...,...
155,YOPOUGON NIANGON SUD,2,26,32.500000,2
156,YOPOUGON SAINT ANDRE,1,3,3.750000,2
157,YOPOUGON SAINT ANDRE,2,77,96.250000,2
158,YOPOUGON ZONE INDUSTRIELLE,1,21,26.582278,3


,Agence,Nombre caisse,nbre,%,profil


In [223]:
caisse = tous.drop_duplicates(subset=['Agence','Numero caisse']).groupby('Agence').size().reset_index().rename(index=str, columns={0:'nbre'})

In [137]:
# test = tous.copy()

In [349]:
# b.groupby(['Agence','Bucket','profil','Periode','TypeOperation'])['NombreTransaction'].mean().reset_index().head(50)

### DIVISION DU FICHIER PAR PROFIL

In [496]:
# ### PROFIL 1 CAISSE
# profil1 = tous[tous['profil']==1]
# resume1 = profil1.groupby(['Agence','profil'])['NombreTransaction'].mean().reset_index()
# resume1.rename(index=str, columns={0:'NombreTransaction'},inplace=True)
# resume1 = resume1.merge(df, on='profil', how = 'left')
# caisse = tous.drop_duplicates(subset=['Agence','Numero caisse']).groupby('Agence').size().reset_index().rename(index=str, columns={0:'nbre'})
# for i in caisse['Agence'].unique():
#     resume1.loc[resume1['Agence']==i,'seuil'] = resume1.loc[resume1['Agence']==i,'seuil'].values/(caisse.loc[caisse['Agence']==i]['nbre'][0])
resume1 = agence[agence['profil']==1]
resume1

,Agence,TypeOperation,profil,NombreTransaction,seuil
0,ABATTA,RETRAIT,1.0,60.645570,38.291035
1,ABOBO SAMAKE,RETRAIT,1.0,37.612500,38.291035
2,ADZOPE,RETRAIT,1.0,32.259259,38.291035
3,AGENCE AKWABA,RETRAIT,1.0,21.519481,38.291035
4,AGENCE ANYAMA,RETRAIT,1.0,32.739726,38.291035
5,AGENCE PRIVILEGE,RETRAIT,1.0,31.691358,38.291035
6,AGHIEN,RETRAIT,1.0,77.567901,38.291035
7,AGNIBILEKROU,RETRAIT,1.0,27.500000,38.291035
8,BANQUE PATRIMONIALE,RETRAIT,1.0,3.351852,38.291035
9,BONOUA,RETRAIT,1.0,43.743590,38.291035


In [497]:
# ### PROFIL 2 CAISSE
# profil2 = tous[tous['profil']==2]
# resume2 = profil2.groupby(['Agence','profil'])['NombreTransaction'].mean().reset_index()
# resume2.rename(index=str, columns={0:'NombreTransaction'},inplace=True)
# resume2 = resume2.merge(df, on ='profil', how = 'left')
# caisse = tous.drop_duplicates(subset=['Agence','Numero caisse']).groupby('Agence').size().reset_index().rename(index=str, columns={0:'nbre'})
# for i in caisse['Agence'].unique():
#     resume2.loc[resume2['Agence']==i,'seuil'] = resume2.loc[resume2['Agence']==i,'seuil'].values/(caisse.loc[caisse['Agence']==i]['nbre'][0])
resume2 = agence[agence['profil']==2]
resume2

,Agence,TypeOperation,profil,NombreTransaction,seuil
60,ABENGOUROU,RETRAIT,2.0,88.049383,74.856646
61,ABOISSO,RETRAIT,2.0,118.086420,74.856646
62,ADJAME,RETRAIT,2.0,47.329268,74.856646
63,ADJAME-LIBERTE,RETRAIT,2.0,46.225000,74.856646
64,AGBOVILLE,RETRAIT,2.0,81.582278,74.856646
...,...,...,...,...,...
127,TREICHVILLE NANAN YAMOUSSO,VERSEMENT,2.0,52.256098,74.618302
128,WILLIAMSVILLE,VERSEMENT,2.0,52.202532,74.618302
129,YOPOUGON NIANGON NORD,VERSEMENT,2.0,91.273973,74.618302
130,YOPOUGON SAINT ANDRE,VERSEMENT,2.0,62.850000,74.618302


In [498]:
# ### PROFIL 3 CAISSE
# profil3 = tous[tous['profil']==3]
# resume3 = profil3.groupby(['Agence','profil'])['NombreTransaction'].mean().reset_index()
# resume3.rename(index=str, columns={0:'NombreTransaction'},inplace=True)
# resume3 = resume3.merge(df, on ='profil', how = 'left')
# caisse = tous.drop_duplicates(subset=['Agence','Numero caisse']).groupby('Agence').size().reset_index().rename(index=str, columns={0:'nbre'})
# for i in caisse['Agence'].unique():
#     resume3.loc[resume3['Agence']==i,'seuil'] = resume3.loc[resume3['Agence']==i,'seuil'].values/(caisse.loc[caisse['Agence']==i]['nbre'][0])
resume3 = agence[agence['profil']==3]
resume3

,Agence,TypeOperation,profil,NombreTransaction,seuil
132,ABOBO GARE,RETRAIT,3.0,70.320988,111.039773
133,AGENCE TOTAL,RETRAIT,3.0,62.135802,111.039773
134,BOUAKE,RETRAIT,3.0,209.592593,111.039773
135,COCODY 2 PLATEAUX,RETRAIT,3.0,118.650000,111.039773
136,DALOA,RETRAIT,3.0,159.382716,111.039773
137,DIVO,RETRAIT,3.0,108.308642,111.039773
138,KOUMASSI SAINT ETIENNE,RETRAIT,3.0,56.085366,111.039773
139,RIVIERA PALMERAIE,RETRAIT,3.0,129.887500,111.039773
140,VRIDI,RETRAIT,3.0,103.222222,111.039773
141,YAMOUSSOUKRO,RETRAIT,3.0,93.901235,111.039773


In [499]:
# ### PROFIL 4 CAISSE
# profil4 = tous[tous['profil']==4]
# resume4 = profil4.groupby(['Agence','profil'])['NombreTransaction'].mean().reset_index()
# resume4.rename(index=str, columns={0:'NombreTransaction'},inplace=True)
# resume4 = resume4.merge(df, on ='profil', how = 'left')
# caisse = tous.drop_duplicates(subset=['Agence','Numero caisse']).groupby('Agence').size().reset_index().rename(index=str, columns={0:'nbre'})
# for i in caisse['Agence'].unique():
#     resume4.loc[resume4['Agence']==i,'seuil'] = resume4.loc[resume4['Agence']==i,'seuil'].values/(caisse.loc[caisse['Agence']==i]['nbre'][0])
resume4 = agence[agence['profil']==4]
resume4

,Agence,TypeOperation,profil,NombreTransaction,seuil
154,AUTOROUTE,RETRAIT,4.0,109.787500,125.950413
155,SAN-PEDRO,RETRAIT,4.0,179.469136,125.950413
156,YOPOUGON BEL AIR,RETRAIT,4.0,88.395062,125.950413
157,AUTOROUTE,VERSEMENT,4.0,70.650000,95.962810
158,SAN-PEDRO,VERSEMENT,4.0,133.987654,95.962810
159,YOPOUGON BEL AIR,VERSEMENT,4.0,82.938272,95.962810


In [308]:
### PROFIL 5 CAISSE
# profil5 = b[b['profil']==5]
# resume5 = profil5.groupby(['Agence','Numero caisse','Bucket','profil','Periode','TypeOperation'])['NombreTransaction'].mean().reset_index()
# resume5.rename(index=str, columns={0:'NombreTransaction'},inplace=True)
# resume5 = resume5.merge(df, on =['Bucket','profil','Periode','TypeOperation'], how = 'left')
# resume5

In [500]:
reporting_path ='D:/Users/11561/Downloads/cash_cobra/'
listeProfil = [resume1,resume2,resume3,resume4]
sheet_names=['Agence 1 Caisse','Agence 2 Caisse','Agence 3 Caisse','Agence 4 Caisse']

In [501]:
with pd.ExcelWriter(reporting_path+"filiere_cash_agence_glob_typeOperation.xlsx") as writer:
    for i in sheet_names:
        listeProfil[sheet_names.index(i)].to_excel(writer, sheet_name=i,index=False)